In [ ]:
import pandas as pd
import numpy as np

import yfinance as yf

from sklearn.svm import LinearSVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

class Config:
    OUTPUT_WINDOW_SIZE = 30
    INPUT_WINDOW_SIZE = 60
    RANDOM_STATE = 7575

In [ ]:
yndx= yf.Ticker("yndx")
data = yndx.history(start="2011-05-24", end="2016-12-31", interval='1d').Close

In [ ]:
def transform_to_DOIU(prices):
    arr = pd.DataFrame(prices)[0].pct_change().to_numpy()[1:]
    x = [1]
    for el in arr:
        x.append(x[-1]*(1+el))
    return np.array(x)

In [ ]:
def prepare_data(prices, output_window_size = Config.OUTPUT_WINDOW_SIZE,
                 input_window_size = Config.INPUT_WINDOW_SIZE):
    X = []
    y = []
    for i in range(0, len(prices)-output_window_size-input_window_size,
                   output_window_size):

        tr_x = transform_to_DOIU(prices[i:input_window_size+i
                                        + output_window_size])
        y.append(tr_x[-1].reshape(1, -1))
        X.append(tr_x[:input_window_size].reshape(1,-1))

    X = np.concatenate(X, axis=0)
    y = np.concatenate(y, axis=0)
    return X, y

def train_model(model, X, y, validate=True, random_state=Config.RANDOM_STATE):
    if validate:
        X_train, X_test, y_train, y_test = train_test_split(X,
                                                            y,
                                                            random_state=random_state)
    else:
        X_train = X
        y_train = y

    model.fit(X_train, y_train)

    if validate:
        y_pred = model.predict(X_test)
        return model, mean_squared_error(y_test, y_pred, squared=False)
    else:
        return model

In [ ]:
X, y = prepare_data(data.values)
model = LinearSVR(max_iter=10000)
best_model, rmse = train_model(model, X, y.ravel())

In [ ]:
rmse

In [ ]:
import pickle
with open('svr.pkl', 'wb') as fid:
    pickle.dump(best_model, fid)

## Creating predictions

In [ ]:
def transform_to_DOIU(prices):
    arr = pd.DataFrame(prices)[0].pct_change().to_numpy()[1:]
    x = [1]
    for el in arr:
        x.append(x[-1] * (1 + el))
    return np.array(x)

In [ ]:
def create_predictions(time_series, bin_path):
    '''
    :param time_series: np.ndarray
    :param bin_path: str
    :return: np.ndarray

    Models are fit with 30 items window.
    '''
    time_series = transform_to_DOIU(time_series).reshape(1, -1)
    with open(bin_path, 'rb') as fid:
        model = pickle.load(fid)
    return model.predict(time_series)

In [ ]:
X_test = data.values[:60]

In [ ]:
create_predictions(X_test, 'svr.pkl')